Самостоятельно реализуйте DecisionTree с некоторыми ограничениями (перечислены ниже). Задание может быть оценено частично.

Он должен получать на вход матрицу объект-признак и целевую переменную. На каждом этапе должен производить определение оптимальной пары признак-величина (для создания разделения данных, поступавших в вершину). Ограничение обучения реализуйте через ограничение допустимой максимальной глубины дерева - остальные опционарно.

**Рассмотрим игрушечную задачу бинарной классификации: поедет ли с Вами новый знакомый из бара? Это будет зависеть от Вашей внешности и красноречия, крепости предлагаемых напитков и, как это ни меркантильно, от количества потраченных в баре денег.**

### Создание набора данных

In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np


# Создание датафрейма с dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis=1)
    out.drop(feature_list, axis=1, inplace=True)
    return out


# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
def intersect_features(train, test):
    common_feat = list(set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [2]:
features = ["Внешность", "Алкоголь_в_напитке", "Уровень_красноречия", "Потраченные_деньги"]
target = ["Поедет"]

**Обучающая выборка**

In [3]:
df_train = {}
df_train["Внешность"] = [
    "приятная",
    "приятная",
    "приятная",
    "отталкивающая",
    "отталкивающая",
    "отталкивающая",
    "приятная",
]
df_train["Алкоголь_в_напитке"] = ["да", "да", "нет", "нет", "да", "да", "да"]
df_train["Уровень_красноречия"] = [
    "высокий",
    "низкий",
    "средний",
    "средний",
    "низкий",
    "высокий",
    "средний",
]
df_train["Потраченные_деньги"] = ["много", "мало", "много", "мало", "много", "много", "много"]
df_train["Поедет"] = LabelEncoder().fit_transform(["+", "-", "+", "-", "-", "+", "+"])

df_train = create_df(df_train, features)
df_train

,Поедет,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_низкий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,False,True,True,False,True,False,False,False,True
1,1,False,True,True,False,False,True,False,True,False
2,0,False,True,False,True,False,False,True,False,True
3,1,True,False,False,True,False,False,True,True,False
4,1,True,False,True,False,False,True,False,False,True
5,0,True,False,True,False,True,False,False,False,True
6,0,False,True,True,False,False,False,True,False,True


**Тестовая выборка**

In [4]:
df_test = {}
df_test["Внешность"] = ["приятная", "приятная", "отталкивающая"]
df_test["Алкоголь_в_напитке"] = ["нет", "да", "да"]
df_test["Уровень_красноречия"] = ["средний", "высокий", "средний"]
df_test["Потраченные_деньги"] = ["много", "мало", "много"]
df_test = create_df(df_test, features)
df_test

,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,False,True,False,True,False,True,False,True
1,False,True,True,False,True,False,True,False
2,True,False,True,False,False,True,False,True


In [5]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y = df_train["Поедет"]
df_train, df_test = intersect_features(train=df_train, test=df_test)

df_train

,Внешность_приятная,Алкоголь_в_напитке_нет,Потраченные_деньги_много,Алкоголь_в_напитке_да,Потраченные_деньги_мало,Уровень_красноречия_средний,Уровень_красноречия_высокий,Внешность_отталкивающая
0,True,False,True,True,False,False,True,False
1,True,False,False,True,True,False,False,False
2,True,True,True,False,False,True,False,False
3,False,True,False,False,True,True,False,True
4,False,False,True,True,False,False,False,True
5,False,False,True,True,False,False,True,True
6,True,False,True,True,False,True,False,False


In [6]:
df_test

,Внешность_приятная,Алкоголь_в_напитке_нет,Потраченные_деньги_много,Алкоголь_в_напитке_да,Потраченные_деньги_мало,Уровень_красноречия_средний,Уровень_красноречия_высокий,Внешность_отталкивающая
0,True,True,True,False,False,True,False,False
1,True,False,False,True,True,False,True,False
2,False,False,True,True,False,True,False,True


Для облегчения рекомендуется использовать (и реализовать) функции приведённые ниже.

In [7]:
# расчёт энтропии множества
def entropy(a_list):
    """
    Calculates and returns entropy of given binary
    1d collection, using
    entropy = -p_0*log2(p_0) - p_1*log2(p_1),
    where
        p_0 - proportion of value 0 in a_list,
        p_1 - proportion of value 1 in a_list,
    """
    
    arr = np.array(a_list)
    n = len(arr)
    if n == 0:
        return 0

    p1 = np.sum(arr == 1) / n
    p0 = np.sum(arr == 0) / n
    
    if p1 == 0 or p0 == 0:
        return 0
    
    return -p0  * np.log2(p0) - p1 * np.log2(p1)

In [14]:
# расчет прироста информации


def information_gain(root, left, right):
    """
    Calculates and returns Information Gain
    if we split root by left and right
    """

    n = len(root)
    if n == 0:
        return 0

    return entropy(root) - ((len(left) / n) * entropy(left) + (len(right) / n) * entropy(right))


information_gain([0, 1, 0, 1, 1, 1, 1, 0], [0, 1, 0, 0], [1, 1, 1, 1])

np.float64(0.5487949406953987)

In [9]:
# определение оптимального разбиения с точки зрения прироста информации


def best_feature_to_split(X, y, to_print=True):
    """Выводит прирост информации при разбиении по каждому признаку"""
    best_ig = -1
    best_feat = None

    for feat in X.columns:
        mask_left = X[feat] == 0
        mask_right = X[feat] == 1

        target_left = y[mask_left]
        target_right = y[mask_right]

        if len(target_left) == 0 or len(target_right) == 0:
            continue

        ig = information_gain(y, target_left, target_right)

        if to_print:
            print(f"feature: {feat}, IG: {ig:.4f}")

        if ig > best_ig:
            best_ig = ig
            best_feat = feat

    return best_feat


best_feature_to_split(df_train, y)

feature: Внешность_приятная, IG: 0.1281
feature: Алкоголь_в_напитке_нет, IG: 0.0060
feature: Потраченные_деньги_много, IG: 0.4696
feature: Алкоголь_в_напитке_да, IG: 0.0060
feature: Потраченные_деньги_мало, IG: 0.4696
feature: Уровень_красноречия_средний, IG: 0.0202
feature: Уровень_красноречия_высокий, IG: 0.2917
feature: Внешность_отталкивающая, IG: 0.1281


'Потраченные_деньги_много'

In [10]:
class DecisionTree:
    """
    Decision tree, that considers binary features and solves
    binary classification task
    """

    class Node:
        """
        Node which are used to build DecisionTree
        """

        def __init__(self, is_leaf=False, val=None, feature=None, depth=None):
            """
            Initalizes a new tree node instance

            Parameters
            ----------
            is_leaf, Boolean
                defines whethes it's leaf node (has no children)
            val, int
                prediction value to be stored if it's leaf node
            feature, str
                name of feature which is used to split data
                by this node
            depth, int
                depth level of this node
            """
            self.left = None
            self.right = None
            self.is_leaf = is_leaf
            self.val = val
            self.feature = feature
            self.depth = depth

    def __init__(self, max_depth=3):
        """
        Initalizes a new DecisionTree instance
        """
        self.max_depth = max_depth
        self.root = None

    def build_tree(self, X, y, curr_depth=0) -> Node:
        """
        Returns recursively built classification tree
        """            
        if len(np.unique(y)) == 1:
            return self.Node(is_leaf=True, val=y.iloc[0], depth=curr_depth)

        if curr_depth >= self.max_depth or X.shape[1] == 0:
            val = y.mode()[0]
            return self.Node(is_leaf=True, val=val, depth=curr_depth)

        best_feature = best_feature_to_split(X, y, to_print=False)

        if not best_feature:
            val = y.mode()[0]
            return self.Node(is_leaf=True, val=val, depth=curr_depth)

        node = self.Node(is_leaf=False, feature=best_feature, depth=curr_depth)

        mask_right = X[best_feature] == 1
        mask_left = X[best_feature] == 0

        X_right, y_right = X[mask_right], y[mask_right]
        X_left, y_left = X[mask_left], y[mask_left]

        node.right = self.build_tree(X_right, y_right, curr_depth + 1)
        node.left = self.build_tree(X_left, y_left, curr_depth + 1)

        return node

    def fit(self, X, y):
        """
        Fits to X: builds a decision tree that is
        appropriate to X
        Returns self.
        """
        self.root = self.build_tree(X, y)
        return self

    def predict(self, X):
        """
        Predicts binary target feature setting off from a tree
        that is built at 'fit' step
        Returns list of predicted values.
        """
        predictions = []
        for i in range(len(X)):
            row = X.iloc[i]
            curr = self.root

            while not curr.is_leaf:
                if row[curr.feature] == 1:
                    if curr.right:
                        curr = curr.right
                    else:
                        break

                else:
                    if curr.left:
                        curr = curr.left
                    else:
                        break

            predictions.append(curr.val)

        return predictions

In [11]:
tree_object = DecisionTree()

In [12]:
tree_object.fit(df_train, y)

In [13]:
tree_object.predict(df_test)

[np.int64(0), np.int64(1), np.int64(1)]